# Python - PyTest

- https://docs.pytest.org/en/latest/
- https://www.youtube.com/watch?time_continue=383&v=pX1_I_sEi8k
- [Effective Python Testing With Pytest](https://realpython.com/pytest-python-testing/)

## Arrange-Act-Assert model:

- **Arrange**, or set up, the conditions for the test
- **Act** by calling some function or method
- **Assert** that some end condition is true

In [24]:
%config Completer.use_jedi = False

import pytest
import ipytest
ipytest.autoconfig()

In [12]:
%%run_pytest[clean]

def test_uppercase():
    assert "loud noises".upper() == "LOUD NOISES"

def test_reversed():
    assert list(reversed([1, 2, 3, 4])) == [4, 3, 2, 1]

def test_some_primes():
    assert 37 in {
        num
        for num in range(1, 50)
        if num != 1 and not any([num % div == 0 for div in range(2, num)])
    }

...                                                                  [100%]
3 passed in 0.05s


## State and Dependency Management -- `fixtures`

- `pytest` fixtures are functions that create data or test doubles or initialize some system state for the test suite.
- Any test that wants to use a fixture must explicitly accept it as an argument, so dependencies are always stated up front.

In [33]:
%%run_pytest[clean] -s

def format_data_for_display(people):
    # Implement this!
    out = []
    for person in people:
        display = f'{person["given_name"]} {person["family_name"]}: {person["title"]}'
        out.append(display)
        
    return out


def format_data_for_excel(people):
    header = "given,family,title"
    out = [header]
    for person in people:
        display = f'{person["given_name"]},{person["family_name"]},{person["title"]}'
        out.append(display)

    out = "\n".join(out) + "\n"
    return out


# def test_format_data_for_display():
#     people = [
#         {
#             "given_name": "Alfonsa",
#             "family_name": "Ruiz",
#             "title": "Senior Software Engineer",
#         },
#         {
#             "given_name": "Sayid",
#             "family_name": "Khan",
#             "title": "Project Manager",
#         },
#     ]

#     assert format_data_for_display(people) == [
#         "Alfonsa Ruiz: Senior Software Engineer",
#         "Sayid Khan: Project Manager",
#     ]


@pytest.fixture
def example_people_data():
    return [
        {
            "given_name": "Alfonsa",
            "family_name": "Ruiz",
            "title": "Senior Software Engineer",
        },
        {
            "given_name": "Sayid",
            "family_name": "Khan",
            "title": "Project Manager",
        },
    ]


def test_format_data_for_display(example_people_data):
    assert format_data_for_display(example_people_data) == [
        "Alfonsa Ruiz: Senior Software Engineer",
        "Sayid Khan: Project Manager",
    ]

def test_format_data_for_excel(example_people_data):
    assert format_data_for_excel(example_people_data) == """given,family,title
Alfonsa,Ruiz,Senior Software Engineer
Sayid,Khan,Project Manager
"""

..
2 passed in 0.01s


#### When to Avoid Fixtures?

Fixtures are great for extracting data or objects that you use across multiple tests. They aren’t always as good for tests that require slight variations in the data. Littering your test suite with fixtures is no better than littering it with plain data or objects. It might even be worse because of the added layer of indirection.

As with most abstractions, it takes some practice and thought to find the right level of fixture use.


#### Fixtures at Scale

You can move fixtures from test modules into more general fixture-related modules. That way, you can import them back into any test modules that need them. This is a good approach when you find yourself using a fixture repeatedly throughout your project.

`pytest` looks for `conftest.py` modules throughout the directory structure. Each `conftest.py` provides configuration for the file tree `pytest` finds it in. You can use any fixtures that are defined in a particular `conftest.py` throughout the file’s parent directory and in any subdirectories. This is a great place to put your most widely used fixtures.

#### monkeypatch
https://docs.pytest.org/en/latest/monkeypatch.html

The monkeypatch fixture helps you to safely set/delete an attribute, dictionary item or environment variable, or to modify `sys.path` for importing.

```py
monkeypatch.setattr(obj, name, value, raising=True)
monkeypatch.delattr(obj, name, raising=True)
monkeypatch.setitem(mapping, name, value)
monkeypatch.delitem(obj, name, raising=True)
monkeypatch.setenv(name, value, prepend=False)
monkeypatch.delenv(name, raising=True)
monkeypatch.syspath_prepend(path)
monkeypatch.chdir(path)
```

In [40]:
%%script false --no-raise-error
# conftest.py

import pytest
import requests

@pytest.fixture(autouse=True)
def disable_network_calls(monkeypatch):
    """
    By placing disable_network_calls() in conftest.py and adding the autouse=True option,
    you ensure that network calls will be disabled in every test across the suite.
    Any test that executes code calling requests.get() will raise a RuntimeError indicating
    that an unexpected network call would have occurred.
    """
    def stunted_get():
        raise RuntimeError("Network access not allowed during testing!")
    monkeypatch.setattr(requests, "get", lambda *args, **kwargs: stunted_get())

## Test Filtering

As your test suite grows, you may find that you want to run just a few tests on a feature and save the full suite for later. pytest provides a few ways of doing this:

- **Name-based filtering**: You can limit pytest to running only those tests whose fully qualified names match a particular expression. You can do this with the `-k` parameter.
- **Directory scoping**: By default, pytest will run only those tests that are in or under the current directory.
- **Test categorization**: pytest can include or exclude tests from particular categories that you define. You can do this with the `-m` parameter.